# LangChain

In [ ]:
!pip install langchain langchain-openai

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("openai_api_key")

In [ ]:
import requests
import nltk
import json
import subprocess
from nltk import sent_tokenize
from bs4 import BeautifulSoup
from typing import Annotated
from pydantic import BaseModel, AfterValidator
from enum import StrEnum, auto
from typing import Type
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
OpenAI?

## Modelos Generativos

In [ ]:
model = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=1.0,
    max_tokens=100
)

In [ ]:
seed = """
#include <stdio.h>

int sum(a int, b int) {
    return a + b;
}

int main(void) {
    int a = 1;
    int b = 2;
    printf("""
result = model.invoke(seed)
print(seed + result)


#include <stdio.h>

int sum(a int, b int) {
    return a + b;
}

int main(void) {
    int a = 1;
    int b = 2;
    printf( "The sum is %d", sum(a, b));
    return 0;
}

output:

The sum is 3


In [ ]:
seed = """
[SYSTEM]
You're an expert chess player. You must answer questions about it.

[HUMAN]
Hello, my name is Juan.

[AGENT]
Hello, my name is Bot. What do you need about chess?

[HUMAN]
What is a Sicilian defense?

[AGENT]"""

result = model.invoke(seed)
print(seed + result)


[SYSTEM]
You're an expert chess player. You must answer questions about it.

[HUMAN]
Hello, my name is Juan.

[AGENT]
Hello, my name is Bot. What do you need about chess?

[HUMAN]
What is a Sicilian defense?

[AGENT]
A Sicilian defense is a popular opening strategy in chess, where Black responds to White's move of 1.e4 with 1...c5. This allows Black to control the center and develop their pieces quickly. It also creates asymmetry in the position, making it harder for White to apply their usual tactics. The Sicilian defense is known for its versatility and complexity, and has been used by top Grandmasters in high-level tournaments. 


## Modelos de Chat

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
model.invoke([
    SystemMessage(content="You're an expert chess player. You must answer questions about it."),
    HumanMessage(content="Hello, my name is Juan."),
    AIMessage(content="Hello, my name is Bot. What do you need about chess?"),
    HumanMessage(content="What is a Sicilian defense?")
])

AIMessage(content="The Sicilian Defense is one of the most popular and aggressive responses to 1.e4, the King's Pawn opening. It starts with the moves:\n\n1. e4 c5\n\nIn this defense, Black aims to create an asymmetrical pawn structure, allowing for counterplay and dynamic chances. The Sicilian Defense has many variations, each leading to different types of positions. Some of the most well-known variations include:\n\n- **Open Sicilian** (after 2.Nf3 and 3.d4)\n- **Najdorf Variation** (5...a6)\n- **Dragon Variation** (5...g6)\n- **Scheveningen Variation** (with d6 and e6)\n- **Sveshnikov Variation** (with 2...e5 on the second move)\n\nThe Sicilian Defense is favored by many players for its fighting potential and has been played at the highest levels of chess throughout history.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 187, 'prompt_tokens': 60, 'total_tokens': 247, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio

In [ ]:
AIMessage(content="Hello, I'm Peter", name="Peter")

AIMessage(content="Hello, I'm Peter", additional_kwargs={}, response_metadata={}, name='Peter')

In [ ]:
(
    "hello"
    "world"
)

'helloworld'

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke([
    SystemMessage(
        content="Eres un estudiante de machine learning, te llamas Lucas y vas "
        "a hacerle preguntas a un profesor"
        ),
    HumanMessage(content="Que es una red neuronal?"),
    AIMessage(
        content="Un modelo bioinspirado que se basa en gradientes, alguna otra pregunta?",
        name="Teacher"
        )
])
response.name = "Lucas"
response

AIMessage(content='Gracias, profesor. ¿Podría explicarme un poco más sobre cómo funcionan las redes neuronales y cuáles son sus componentes principales?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 61, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CdPS8vHdp7KvhKWDHjxqIlWoMiwIF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, name='Lucas', id='run--56b49c7e-3c72-4b0c-8817-6726497c6d02-0', usage_metadata={'input_tokens': 61, 'output_tokens': 26, 'total_tokens': 87, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- Zero shot

In [ ]:
class ClassificationOptions(StrEnum):
    POSITIVE = auto()
    NEUTRAL = auto()
    NEGATIVE = auto()

In [ ]:
ClassificationOptions.POSITIVE

<ClassificationOptions.POSITIVE: 'positive'>

In [ ]:
def generate_prompt(categories: Type[StrEnum], description: str):
    return (f"Te encargas de clasificar texto segun {description}, puedes responder"
        f" unicamente los siguientes valores: [{', '.join(list(categories))}]. No debes responder "
        "nada adicional a esos dos valores. No agregar ninguna explicacion ni texto adicional.")

In [ ]:
generate_prompt(ClassificationOptions, description="sentimientos")

'Te encargas de clasificar texto segun sentimientos, puedes responder unicamente los siguientes valores: [positive, neutral, negative]. No debes responder nada adicional a esos dos valores. No agregar ninguna explicacion ni texto adicional.'

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke([
    SystemMessage(content=generate_prompt(ClassificationOptions, "sentimientos")),
    HumanMessage(content="Me siento triste :("),
])
validated_output = ClassificationOptions(response.content)
validated_output

<ClassificationOptions.NEGATIVE: 'negative'>

In [ ]:
class SportsOptions(StrEnum):
    SOCCER = auto()
    BASQUETBALL = auto()

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke([
    SystemMessage(content=generate_prompt(SportsOptions, "deportes")),
    HumanMessage(content="Tiro de esquina"),
])
validated_output = SportsOptions(response.content)
validated_output

<SportsOptions.SOCCER: 'soccer'>

In [ ]:
class ParityOptions(StrEnum):
    PAR = auto()
    IMPAR = auto()

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke([
    SystemMessage(content=generate_prompt(ParityOptions, "numero par")),
    HumanMessage(content="2"),
])
validated_output = ParityOptions(response.content)
validated_output

<ParityOptions.PAR: 'par'>

- few shot

In [ ]:
class SentimentEnum(StrEnum):
    POSITIVE = auto()
    NEUTRAL = auto()
    NEGATIVE = auto()

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke([
    SystemMessage(content=generate_prompt(SentimentEnum, "sentimientos")),
    # 1-shot example
    HumanMessage(content="Me siento triste"),
    AIMessage(content="negative"),
    # 2-shot example
    HumanMessage(content="Me siento feliz"),
    AIMessage(content="positive"),
    # ---
    HumanMessage(content="No se como me siento.")
])
validated_output = SentimentEnum(response.content)
validated_output

<SentimentEnum.NEUTRAL: 'neutral'>

In [ ]:
def validate_month(month: int) -> int:
    if month < 1 or month > 12:
        raise ValueError(f"{month} is not a valid month.")
    return month

def validate_day(day: int) -> int:
    if day < 1 or day > 31:
        raise ValueError(f"{day} is not a valid day.")
    return day

class Date(BaseModel):
    year: int
    month: Annotated[int, AfterValidator(validate_month)]
    day: Annotated[int, AfterValidator(validate_day)]

In [ ]:
Date(year=2025, month=1, day=50)

ValidationError: 1 validation error for Date
day
  Value error, 50 is not a valid day. [type=value_error, input_value=50, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke([
    SystemMessage(content="Te encargas de estructurar fechas en formato json."
    " No importa que sean fechas invalidas, siempre devuelve los campos de year, month y day."),
    # 1-shot example
    HumanMessage(content="primero de agosto de 2025"),
    AIMessage(content='{"year": 2025, "month": 8, "day": 1}'),
    # ---
    HumanMessage(content="28 de febrero de 2005")
])
validated_output = Date.model_validate_json(response.content)
validated_output

Date(year=2005, month=2, day=28)

In [ ]:
validated_output.day

28

## Embeddings

In [ ]:
embedding_model = OpenAIEmbeddings()

In [ ]:
print(embedding_model)

client=<openai.resources.embeddings.Embeddings object at 0x7eda78ddfc20> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7ed980f76e70> model='text-embedding-ada-002' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


In [ ]:
result = embedding_model.embed_documents(["hello, I'm Juan"])

In [ ]:
type(result)

list

In [ ]:
type(result[0])

list

In [ ]:
len(result[0])

1536

## Retrieval Augmented Generation (RAG)

In [ ]:
# response = requests.get("https://es.wikipedia.org/wiki/Alan_Turing")
with open("turing.html") as f:
    html = f.read()

In [ ]:
html[:1000]

'<!DOCTYPE html>\n<!-- saved from url=(0041)https://en.wikipedia.org/wiki/Alan_Turing -->\n<html class="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available vector-animations-ready ve-available" lang="en" dir="ltr"><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\n<title>Alan Turing - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature

In [ ]:
soup = BeautifulSoup(html)

In [ ]:
text = soup.find(id="bodyContent").get_text()
print(text[:1000])







From Wikipedia, the free encyclopedia


English computer scientist (1912–1954)
"Turing" redirects here. For other uses, see Turing (disambiguation).


Alan TuringOBE FRSTuring in 1951BornAlan Mathison Turing(1912-06-23)23 June 1912Maida Vale, London, EnglandDied7 June 1954(1954-06-07) (aged 41)Wilmslow, Cheshire, EnglandCause of deathCyanide poisoning as an act of suicide[note 1]Alma mater
University of Cambridge  (MA)
Princeton University (PhD)
Known for
Cryptanalysis of the Enigma
Turing's proof
Turing machine
Turing test
unorganised machine
Turing pattern
Turing reduction
"The Chemical Basis of Morphogenesis"
Turing paradox
AwardsSmith's Prize (1936)Scientific careerFields
Logic
mathematics
cryptanalysis
computer science
mathematical and theoretical biology[1]
Institutions
University of Manchester
Government Code and Cypher School
National Physical Laboratory
ThesisSystems of Logic Based on Ordinals (1938)Doctoral advisorAlonzo Church[2]Doctoral students
Robin Gandy[2][3]
Beat

In [ ]:
tokens = text.split("\n\n")

In [ ]:
len(tokens)

291

In [ ]:
documents = [Document(page_content=token.strip()) for token in tokens]

In [ ]:
vector_db = InMemoryVectorStore(embedding=OpenAIEmbeddings())

In [ ]:
vector_db.add_documents(documents)

['d016f13f-c1b1-4eba-9206-0503f2df643f',
 'badb4cb7-7167-4edb-a7ea-f8c3d2436263',
 'ee7a1816-34bf-440d-a953-24cbc9dd46af',
 '17076e00-6e31-4c4c-bce2-09d50a12d4b2',
 '5051bb17-58d0-407d-858e-2bb2df10bbd2',
 'e57d5e7c-de27-4aa9-9464-9228a5ed20e9',
 '33c47c68-0bf3-438a-8cc0-c2329f46f699',
 '6b167689-3f51-4e1c-b613-bc5b1d9d192e',
 '048a5d4d-cbc3-4152-829c-b79a16f1a18d',
 'a8a04d32-efb9-4fb6-be86-cdc0bfe51c6d',
 '42de9523-b0f9-4547-ba4e-443fb84e7617',
 '2eacd59d-f364-4a72-bdc5-1756b1f24259',
 '8f4e6bb1-bd12-436a-a7a5-02eed364b56c',
 '0be6189d-5a8e-4205-a953-749a9a0502f2',
 '1aea8e5d-7b8c-4d6c-8d8d-9e4152ca470c',
 '6741487a-e14a-4799-af2a-015855f2a0de',
 '2dc011cc-ef41-4733-a72e-41b880c024dc',
 'e75f3a5b-cd1c-46eb-922a-505102dadd83',
 '49041fa9-3abe-4ce9-8f51-dac29412d702',
 '5e1fe013-cd42-4ed8-94c8-3b4bdad8ad55',
 '3b61c4af-d630-4b71-8720-6da443bc27eb',
 'd25f045e-9567-4640-bb44-fe46f1309c00',
 '79e633e8-70d1-425d-b8e5-85ee94bd9365',
 'f8da11e7-fc73-44b6-8a3e-24d52d37bdc8',
 'dad3a73d-f5c6-

In [ ]:
vector_db.similarity_search("when was alan turing born?", k=1)

[Document(id='bb62e1c9-d3d9-4de3-b7b8-0097a5aae7eb', metadata={}, page_content='^ Newman, M.H.A. (1955). "Alan Mathison Turing. 1912–1954". Biographical Memoirs of Fellows of the Royal Society. 1: 253–263. doi:10.1098/rsbm.1955.0019. ISSN\xa00080-4606. JSTOR\xa0769256. S2CID\xa0711366.')]

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")
question = "Did Alan turing die?"
context = "\n".join(
    map(
        lambda doc: "\t- " + doc.page_content,
        vector_db.similarity_search(question, k=3)
        )
    )
model = ChatOpenAI(model="gpt-4o-mini")
response = model.invoke([
    SystemMessage(content=f"""
Based on the following context:

{context}

Answer the user questions.
"""
    ),
    HumanMessage(content=question)
    ])
print(response.content)

Yes, Alan Turing died on June 7, 1954. His death was ruled a suicide, and it is believed that he took his own life by ingesting cyanide.


In [ ]:
100 * len(context) / len(text)

4.016927456487267

## Tools

In [ ]:
def add(a: float, b: float = 1.0) -> float:
    """
    This function sums two numbers.

    :param a: First number.
    :type a: float
    :param b: Second number.
    :type b: float
    """
    return a + b

In [ ]:
tools = [add]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

In [ ]:
# tools2 = [divide]
# model2 = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools2)

In [ ]:
model

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7ed960a33da0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7ed9621f38c0>, root_client=<openai.OpenAI object at 0x7ed95fdbb770>, root_async_client=<openai.AsyncOpenAI object at 0x7ed95fc7a4e0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'This function sums two numbers. :param a: First number.\n:type a: float\n:param b: Second number.\n:type b: float', 'parameters': {'properties': {'a': {'type': 'number'}, 'b': {'default': 1.0, 'type': 'number'}}, 'required': ['a'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [ ]:
response = model.invoke([HumanMessage(content="What is the result of -1388.88181 + 4777.177")])

In [ ]:
response.content

''

In [ ]:
result = []
tool_call = response.tool_calls[0]
observation = str(eval(tool_call["name"])(**tool_call["args"]))
observation

'3388.29519'

In [ ]:
tool_call

{'name': 'add',
 'args': {'a': -1388.88181, 'b': 4777.177},
 'id': 'call_NbPn1SHWaRkFXkJoQFzqg4Bq',
 'type': 'tool_call'}

In [ ]:
messages = [
    HumanMessage(content="What is the result of -1388.88181 + 4777.177"),
    response,
    ToolMessage(content=observation, tool_call_id=tool_call["id"])
]
print(model.invoke(messages).content)

The result of -1388.88181 + 4777.177 is 3388.29519.


In [ ]:
print(subprocess.run(["ls", "-l"], capture_output=True).stdout.decode())

total 884
drwxr-xr-x 1 root root   4096 Nov 17 14:29 sample_data
-rw-r--r-- 1 root root 900669 Nov 19 01:02 turing.html



In [ ]:
def shell_command(command: list[str]) -> str:
    """
    Executes a shell command and returns the stdout.

    :param command: Command to run as a list of words.
    :type command: list[str]
    :return: Stdout of the command.
    :rtype: str
    """
    return subprocess.run(command, capture_output=True).stdout.decode()

In [ ]:
tools = [shell_command]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

In [ ]:
messages = [
    HumanMessage(content="Which files are in my current directory?")
]
response = model.invoke(messages)
messages.append(response)
result = []
tool_call = response.tool_calls[0]
observation = str(eval(tool_call["name"])(**tool_call["args"]))

In [ ]:
observation

'total 888\ndrwxr-xr-x 2 root root   4096 Nov 19 02:01 example\ndrwxr-xr-x 1 root root   4096 Nov 17 14:29 sample_data\n-rw-r--r-- 1 root root 900669 Nov 19 01:02 turing.html\n'

In [ ]:
messages.append(
    ToolMessage(content=observation, tool_call_id=tool_call["id"])
)
print(model.invoke(messages).content)

In your current directory, the following files and folders are present:

1. **Directories:**
   - `example` (last modified: Nov 19, 02:01)
   - `sample_data` (last modified: Nov 17, 14:29)

2. **File:**
   - `turing.html` (last modified: Nov 19, 01:02, size: 900669 bytes)
